In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.stats import multivariate_normal
pd.set_option('display.max_columns', None)  


In [3]:
mypath = 'Datasets/Gaze10minData/'
mydtaframes = {}

for index,files in enumerate(os.listdir(mypath)):
    mydtaframes["df_{}".format(index)] = pd.read_excel('{}/{}'.format(mypath, files),
                                                         sheet_name= 'Sheet1')

In [4]:
mydtaframes['df_1'].head()

,ParticipantName,RecordingDuration,RecordingTimestamp,LocalTimeStamp,EyeTrackerTimestamp,MouseEventIndex,MouseEventX (MCSpx),MouseEventY (MCSpx),KeyPressEventIndex,StudioEventIndex,StudioEventIndex_B,StudioEvent,StudioEvent_B,FixationIndex,SaccadeIndex,GazeEventType,GazeEventType_B,GazeEventDuration,FixationPointX (MCSpx),FixationPointY (MCSpx),AbsoluteSaccadicDirection,RelativeSaccadicDirection,GazePointIndex,GazePointX (MCSpx),GazePointY (MCSpx),PupilLeft,PupilRight,ValidityLeft,ValidityRight,StudioEventTypeDiff,GazeEventTypeDiff
0,Proband_31_D2,2895701,440,13:25:24.097,1.458043e+15,NaN,NaN,NaN,NaN,NaN,NaN,0_unstated,0_unstated,NaN,2.0,Saccade,Saccade,133,NaN,NaN,NaN,NaN,21.0,66.0,220.0,"2,74","2,59",0.0,0.0,Agree_0_unstated,Agree_Saccade
1,Proband_31_D2,2895701,456,13:25:24.114,1.458043e+15,NaN,NaN,NaN,NaN,NaN,NaN,0_unstated,0_unstated,NaN,2.0,Saccade,Saccade,133,NaN,NaN,NaN,NaN,22.0,20.0,166.0,"2,59","2,54",0.0,0.0,Agree_0_unstated,Agree_Saccade
2,Proband_31_D2,2895701,473,13:25:24.131,1.458043e+15,NaN,NaN,NaN,NaN,NaN,NaN,0_unstated,0_unstated,2.0,NaN,Fixation,Fixation,200,45.0,168.0,"339,44",NaN,23.0,20.0,168.0,NaN,"2,62",4.0,0.0,Agree_0_unstated,Agree_Fixation
3,Proband_31_D2,2895701,490,13:25:24.147,1.458043e+15,NaN,NaN,NaN,NaN,NaN,NaN,0_unstated,0_unstated,2.0,NaN,Fixation,Fixation,200,45.0,168.0,"339,44",NaN,24.0,36.0,171.0,"2,58","2,50",0.0,0.0,Agree_0_unstated,Agree_Fixation
4,Proband_31_D2,2895701,506,13:25:24.164,1.458043e+15,NaN,NaN,NaN,NaN,NaN,NaN,0_unstated,0_unstated,2.0,NaN,Fixation,Fixation,200,45.0,168.0,"339,44",NaN,25.0,66.0,224.0,"2,74","2,60",0.0,0.0,Agree_0_unstated,Agree_Fixation


In [5]:
studiodf = {}

for index,frames in enumerate(mydtaframes):
    studiodf["{}".format(index)] = mydtaframes[frames]

In [6]:
len(studiodf)

20

In [7]:
for frames in studiodf:
    studiodf[frames].fillna(method='ffill', inplace= True)

In [8]:
allData = pd.concat([studiodf['0'],studiodf['1'],studiodf['2'],studiodf['3'],studiodf['4'],studiodf['5'],
                    studiodf['6'],studiodf['7'],studiodf['8'],studiodf['9'],studiodf['10'],studiodf['11'],
                    studiodf['12'],studiodf['13'],studiodf['14'],studiodf['15'],studiodf['16'],studiodf['17'],
                    studiodf['18'],studiodf['19'],], sort = False)
allData['FixationPointX (MCSpx)'].fillna(method='bfill', inplace= True)
allData['FixationPointY (MCSpx)'].fillna(method='bfill', inplace= True)

In [9]:
allData['FixationPointX (MCSpx)'] = allData['FixationPointX (MCSpx)'].diff()
allData['FixationPointY (MCSpx)'] = allData['FixationPointY (MCSpx)'].diff()
allData['FixationPointX (MCSpx)'].fillna(method='bfill', inplace= True)
allData['FixationPointY (MCSpx)'].fillna(method='bfill', inplace= True)

In [30]:
import scipy.stats as stats
#Selecting p value as 0.05
#Ho: There is no relation  between variables
#Ha: There is a relationship
#Result p value should be greater than 0.05 to accept null hypothesis

In [35]:
pd.crosstab(allData['StudioEvent'],allData['GazeEventType'])

GazeEventType,Fixation,Saccade,Unclassified
StudioEvent,,,
0_Unstated,350,53,2
0_unstated,30048,7534,5054
1_Scanning,116211,50252,13231
2_Skimming,33232,13461,3682
3_Reading,80487,14911,2579
4_MediaView,3002,975,197
5_Unknown,157357,78184,119557


In [36]:
def drop_unstated(series):
    if series.lower() == '0_Unstated' or series.lower() == '0_unstated':
        return 
    else:
        return series

In [37]:
allData['Studio_Events'] = allData['StudioEvent'].apply(drop_unstated)

In [38]:
studio_crosstab = pd.crosstab(allData['Studio_Events'],allData['GazeEventType'])
studio_crosstab

GazeEventType,Fixation,Saccade,Unclassified
Studio_Events,,,
1_Scanning,116211,50252,13231
2_Skimming,33232,13461,3682
3_Reading,80487,14911,2579
4_MediaView,3002,975,197
5_Unknown,157357,78184,119557


In [39]:
stats.chi2_contingency(studio_crosstab)

(94640.53405167622,
 0.0,
 8,
 array([[102038.05453371,  41251.15070753,  36404.79475876],
        [ 28605.11200783,  11564.25210019,  10205.63589197],
        [ 55635.59422713,  22491.925122  ,  19849.48065088],
        [  2370.17841232,    958.19728568,    845.624302  ],
        [201640.06081901,  81517.4747846 ,  71940.46439639]]))

In [ ]:
#Ho is rejected bc p value is less than 0.05
#There is a relation between Gaze event type and Studio events

In [47]:
#Running the same test with 10% of data

subset_allData = allData.sample(frac=0.1)
len(subset_allData)

73036

In [48]:
subset_allData['Studio_Events'] = subset_allData['StudioEvent'].apply(drop_unstated)

In [50]:
studio_crosstabsub = pd.crosstab(subset_allData['Studio_Events'],subset_allData['GazeEventType'])
studio_crosstabsub

GazeEventType,Fixation,Saccade,Unclassified
Studio_Events,,,
1_Scanning,11711,4909,1294
2_Skimming,3433,1292,376
3_Reading,8149,1465,252
4_MediaView,290,105,18
5_Unknown,15686,7786,12003


In [51]:
stats.chi2_contingency(studio_crosstabsub)

(9664.92635176253,
 0.0,
 8,
 array([[10229.38920153,  4052.52509125,  3632.08570722],
        [ 2912.81200832,  1153.95391819,  1034.23407349],
        [ 5633.7587285 ,  2231.89754104,  2000.34373046],
        [  235.83441667,    93.42932135,    83.73626198],
        [20257.20564499,  8025.19412817,  7192.60022685]]))

In [ ]:
#Results are same, reject Ho hypothesis

In [52]:
df = pd.read_csv("https://raw.githubusercontent.com/Opensourcefordatascience/Data-sets/master//Iris_Data.csv")

In [53]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
